In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.sql import functions as F

In [2]:
# spark = SparkSession.builder.appName("drug-to_target_biodata").getOrCreate()
spark = SparkSession.builder \
    .appName("drug-to_target_biodata_3") \
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/02 18:19:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Biodata from ChEMBL

In [3]:
# Testset with only min_standard_value
biodata_in = "./data/drug2target_bioactivities_chembl_33_grouped.csv"

biodata = spark.read.csv(biodata_in, header=True, inferSchema=True)
biodata = biodata.drop('_c0')
num_rows_biodata = biodata.count()
biodata.show()
print("Number of rows:", num_rows_biodata)

biodata_uni = (biodata.groupBy("chembl_id", "accession")
          .agg(F.count("*").alias("count_of_combinations")))
num_uni_rows_biodata = biodata_uni.count()
print("Number of unique drug-target pairs:", num_uni_rows_biodata)

+------------+--------+------+---------+--------------------+----------------+------------------+--------------+
|   chembl_id|molregno|   tid|accession|           pref_name|target_chembl_id|min_standard_value|standard_units|
+------------+--------+------+---------+--------------------+----------------+------------------+--------------+
|  CHEMBL1000|  111185|104067|   Q96FL8|Multidrug and tox...|   CHEMBL1743126|               2.0|             %|
|  CHEMBL1000|  111185| 17045|   P08684| Cytochrome P450 3A4|       CHEMBL340|              13.0|             %|
|  CHEMBL1000|  111185|   165|   Q12809|                HERG|       CHEMBL240|          30199.52|            nM|
|  CHEMBL1000|  111185|103947|   Q9Y6L6|Solute carrier or...|   CHEMBL1697668|              35.7|             %|
|  CHEMBL1000|  111185|   127|   P35367|Histamine H1 rece...|       CHEMBL231|              5.89|            nM|
|  CHEMBL1000|  111185| 12675|   P02763|Alpha-1-acid glyc...|      CHEMBL4285|              5.12

In [4]:
biodata_uni_t = biodata.dropDuplicates(["target_chembl_id"])
num_uni_t_biodata = biodata_uni_t.count()
print("Number of unique targets with biodata:", num_uni_t_biodata)

Number of unique targets with biodata: 2372


## Target-Drug pairs from OpenTargets (GE, CT, probes)

In [6]:
# gcloud storage rsync gs://ot-team/irene/drug_to_target data/drug_to_target

drug2target_parquet_dir = "./data/drug_to_target"
drug2target_parquet = spark.read.parquet(drug2target_parquet_dir)

drug2target_parquet.show()

num_rows_drug2target = drug2target_parquet.count()
print("Number of rows:", num_rows_drug2target)

+----------+---------+---------------+--------------------+------------------+-------------------+
|    drugId|uniprotId|       targetId|             sources|isHighQualityProbe|isTherapeuticTarget|
+----------+---------+---------------+--------------------+------------------+-------------------+
|CHEMBL1000|   O00167|ENSG00000064655|[ot_genetics_portal]|             false|              false|
|CHEMBL1000|   O00555|ENSG00000141837|[uniprot_literatu...|             false|              false|
|CHEMBL1000|   O14633|ENSG00000159455|[ot_genetics_portal]|             false|              false|
|CHEMBL1000|   O60706|ENSG00000069431|            [chembl]|             false|               true|
|CHEMBL1000|   P00352|ENSG00000165092|[ot_genetics_portal]|             false|              false|
|CHEMBL1000|   P01567|ENSG00000214042|            [chembl]|             false|               true|
|CHEMBL1000|   P04155|ENSG00000160182|[ot_genetics_portal]|             false|              false|
|CHEMBL100

In [7]:
# Remove rows that have duplicate drugId, targetId combinations
drug2target_uni = drug2target_parquet.dropDuplicates(["targetId"])
drug2target_uni.show()

num_rows_uni_drug2target = drug2target_uni.count()
print("Number of unique targets with GE+:", num_rows_uni_drug2target)

+-------------+---------+---------------+--------------------+------------------+-------------------+
|       drugId|uniprotId|       targetId|             sources|isHighQualityProbe|isTherapeuticTarget|
+-------------+---------+---------------+--------------------+------------------+-------------------+
|    CHEMBL423|   Q9NSG2|ENSG00000000460|[ot_genetics_portal]|             false|              false|
|CHEMBL1187417|   Q9Y6X5|ENSG00000001561|              [impc]|             false|              false|
|   CHEMBL6995|   Q9Y6D9|ENSG00000002822|[impc, ot_genetic...|             false|              false|
| CHEMBL193240|   Q9Y216|ENSG00000003987|[ot_genetics_portal]|             false|              false|
|CHEMBL1088752|   P52569|ENSG00000003989|              [impc]|             false|              false|
|CHEMBL2103959|   Q9Y2S7|ENSG00000004142|[impc, ot_genetic...|             false|              false|
|  CHEMBL77622|   Q96JG6|ENSG00000004766|[ot_genetics_portal]|             false| 

Number of unique targets with GE+: 16347


### Biodata and evidence data intersection

In [25]:
columns_to_join = drug2target_parquet.select('drugId', 'uniprotId', 'sources', 'isHighQualityProbe', 'isTherapeuticTarget')

biodata_join = biodata.join(columns_to_join, 
                            (biodata.chembl_id == columns_to_join.drugId) & 
                            (biodata.accession == columns_to_join.uniprotId), 
                            how="inner")

biodata_join.show()

biodata_join_uni = (biodata_join.groupBy("chembl_id", "accession")
          .agg(F.count("*").alias("count_of_combinations")))

num_uni_rows_biodata_join = biodata_join_uni.count()

print("Number of unique drug-target pairs with GE+ and bidata:", num_uni_rows_biodata_join)

print("Number of unique drug-target pairs with bidata:", num_uni_rows_biodata)

print("For", round(num_uni_rows_biodata_join/num_uni_rows_biodata*100), "% of drug-target pairs with with bidata we have GE+")

+-------------+--------+------+---------+--------------------+----------------+------------------+--------------+-------------+---------+--------------------+------------------+-------------------+
|    chembl_id|molregno|   tid|accession|           pref_name|target_chembl_id|min_standard_value|standard_units|       drugId|uniprotId|             sources|isHighQualityProbe|isTherapeuticTarget|
+-------------+--------+------+---------+--------------------+----------------+------------------+--------------+-------------+---------+--------------------+------------------+-------------------+
| CHEMBL103667|  167995| 11636|   P05771|Protein kinase C ...|      CHEMBL3045|            6900.0|            nM| CHEMBL103667|   P05771|[ot_genetics_port...|             false|               true|
|    CHEMBL104|   13494|   130|   P35462|Dopamine D3 receptor|       CHEMBL234|             899.0|            nM|    CHEMBL104|   P35462|[ot_genetics_port...|             false|               true|
|    CHEMB

Number of unique drug-target pairs with GE+ and bidata: 24880
Number of unique drug-target pairs with bidata: 53991
For 46 % of drug-target pairs with with bidata we have GE+


### For how many drugs from GE+ supported list of D-T pairs we have biodata?

In [39]:
drugs_uni = "./data/drug_to_target_unique_drugs.csv"

drugs_uni = spark.read.csv(drugs_uni, header=True, inferSchema=True)
num_rows_drugs_uni = drugs_uni.count()
print("Number of unique drugs from GE+ list:", num_rows_drugs_uni)

biodata_drugs_uni = (biodata_join.groupBy("chembl_id")
          .agg(F.count("*").alias("count_of_combinations")))
num_rows_drugs_uni_biodata = biodata_drugs_uni.count()
print("Number of unique drugs from biodata list:", num_rows_drugs_uni_biodata)

print("For", round(num_rows_drugs_uni_biodata/num_rows_drugs_uni*100), "% of drugs from GE+ list we have biodata")

Number of unique drugs from GE+ list: 12835


Number of unique drugs from biodata list: 6652
For 52 % of drugs from GE+ list we have biodata


### How many targets with GE+ and bioactivity data are not therapeutic targets (not in MoA)?

In [65]:
biodata_join_therapy = biodata_join.filter(biodata_join.isTherapeuticTarget == False)

biodata_join_targets_uni = (biodata_join.groupBy("target_chembl_id")
          .agg(F.count("*").alias("count_of_combinations")))
num_biodata_join_targets_uni = biodata_join_targets_uni.count()

print("Number of unique targets with GE+ & biodata:", num_biodata_join_targets_uni)

biodata_join_therapy_uni = (biodata_join_therapy.groupBy("target_chembl_id")
          .agg(F.count("*").alias("count_of_combinations")))
num_biodata_join_therapy_uni = biodata_join_therapy_uni.count()

print("Number of unique targets with GE+ & biodata & not therapeutic targets:", num_biodata_join_therapy_uni)

print(round(num_biodata_join_therapy_uni/num_biodata_join_targets_uni*100), "% of targets with GE+ & biodata are not therapeutic")

Number of unique targets with GE+ & biodata: 1839


Number of unique targets with GE+ & biodata & not therapeutic targets: 1422
77 % of targets with GE+ & biodata are not therapeutic


### Repeating the targets analysis with pChEMBL value cutoff (pChEMBL >= 5)

In [3]:

# Testset with all biodata
biodata_all_in = "./data/drug2target_bioactivities_chembl_33.csv"

biodata_all = spark.read.csv(biodata_all_in, header=True, inferSchema=True)
biodata_all = biodata_all.drop('_c0')

# Filter rows where pChEMBL is less than or equal to 5
biodata_all = biodata_all.filter(biodata_all['pchembl_value'] >= 5)
biodata_all.show()

biodata_all_count = biodata_all.count()
print("Number of all drug-target pairs with pChEMBL >= 5:", biodata_all_count)

# biodata_all.repartition(1).write.csv("data/biodata_all_join_test", header=True)


+----------+--------+---------------+------+--------------+--------------------+-----------+--------------------+---+---------+--------------------+----------------+------------+--------------+--------------+-----------------+-------------+
| chembl_id|molregno|assay_chembl_id|src_id|src_short_name|             journal|  pubmed_id|                 doi|tid|accession|           pref_name|target_chembl_id|    organism|standard_value|standard_units|standard_relation|pchembl_value|
+----------+--------+---------------+------+--------------+--------------------+-----------+--------------------+---+---------+--------------------+----------------+------------+--------------+--------------+-----------------+-------------+
|CHEMBL1000|  111185|   CHEMBL830377|     1|    LITERATURE|Bioorg Med Chem Lett|1.5686917E7|10.1016/j.bmcl.20...|127|   P35367|Histamine H1 rece...|       CHEMBL231|Homo sapiens|          5.89|            nM|                =|         8.23|
|CHEMBL1000|  111185|  CHEMBL1224926

In [12]:
# !!! Should apply min values but not random!!
from pyspark.sql import Window
from pyspark.sql.functions import row_number, concat_ws

# Define window specification
windowSpec = Window.partitionBy("chembl_id", "target_chembl_id").orderBy(biodata_all["pchembl_value"].desc())

# Assign row number
biodata_all_sort = biodata_all.withColumn("row_num", row_number().over(windowSpec))

# Filter to keep only rows with maximum pchembl_value for each chembl_id within each target_chembl_id
biodata_all_min = biodata_all_sort.filter(biodata_all_sort.row_num == 1).drop("row_num")


biodata_all_min.show()

print("Number of unique drug-target pairs with pChEMBL >= 5:", biodata_all_min.count())


# biodata_all_join_min = biodata_all_join_sort.withColumn("sources", concat_ws(",", biodata_all_join["sources"]))
# biodata_all_join_min.repartition(1).write.csv("data/biodata_all_join_test", header=True)


+------------+--------+---------------+------+--------------+--------------------+-----------+--------------------+------+---------+--------------------+----------------+------------+--------------+--------------+-----------------+-------------+
|   chembl_id|molregno|assay_chembl_id|src_id|src_short_name|             journal|  pubmed_id|                 doi|   tid|accession|           pref_name|target_chembl_id|    organism|standard_value|standard_units|standard_relation|pchembl_value|
+------------+--------+---------------+------+--------------+--------------------+-----------+--------------------+------+---------+--------------------+----------------+------------+--------------+--------------+-----------------+-------------+
|  CHEMBL1000|  111185|   CHEMBL830377|     1|    LITERATURE|Bioorg Med Chem Lett|1.5686917E7|10.1016/j.bmcl.20...|   127|   P35367|Histamine H1 rece...|       CHEMBL231|Homo sapiens|          5.89|            nM|                =|         8.23|
| CHEMBL10041|  

In [15]:
drug2target_parquet_dir = "./data/drug_to_target"
drug2target_parquet = spark.read.parquet(drug2target_parquet_dir)

columns_to_join = drug2target_parquet.select('drugId', 'uniprotId', 'sources', 'isHighQualityProbe', 'isTherapeuticTarget')

biodata_all_join = biodata_all_min.join(columns_to_join, 
                            (biodata_all_min.chembl_id == columns_to_join.drugId) & 
                            (biodata_all_min.accession == columns_to_join.uniprotId), 
                            how="inner")
biodata_all_join.show()

num_biodata_all_join = biodata_all_join.count()
num_biodata_all_min = biodata_all_min.count()

print("Number of unique drug-target pairs with GE+ with pChEMBL >= 5:", num_biodata_all_join)

print("Number of unique drug-target pairs with pChEMBL >= 5:", num_biodata_all_min)

print("For", round(num_biodata_all_join/num_biodata_all_min*100), "% of drug-target pairs with pChEMBL >= 5 we have GE+")

+-------------+--------+---------------+------+--------------+--------------------+-----------+--------------------+------+---------+--------------------+----------------+------------+--------------+--------------+-----------------+-------------+-------------+---------+--------------------+------------------+-------------------+
|    chembl_id|molregno|assay_chembl_id|src_id|src_short_name|             journal|  pubmed_id|                 doi|   tid|accession|           pref_name|target_chembl_id|    organism|standard_value|standard_units|standard_relation|pchembl_value|       drugId|uniprotId|             sources|isHighQualityProbe|isTherapeuticTarget|
+-------------+--------+---------------+------+--------------+--------------------+-----------+--------------------+------+---------+--------------------+----------------+------------+--------------+--------------+-----------------+-------------+-------------+---------+--------------------+------------------+-------------------+
| CHEMB

Number of unique drug-target pairs with GE+ with pChEMBL >= 5: 16138
Number of unique drug-target pairs with pChEMBL >= 5: 26698
For 60 % of drug-target pairs with pChEMBL >= 5 we have GE+


In [16]:
biodata_all_join_targets = biodata_all_join.groupBy("target_chembl_id").count()
num_biodata_all_join_targets = biodata_all_join_targets.count()

print("Number of unique targets with GE+ & pChEMBL >= 5:", num_biodata_all_join_targets)

Number of unique targets with GE+ & pChEMBL >= 5: 1488


In [17]:
biodata_all_join_therapy = biodata_all_join.filter(biodata_all_join.isTherapeuticTarget == False)

biodata_all_join_therapy_uni = biodata_all_join_therapy.groupBy("target_chembl_id").count()
num_biodata_all_join_therapy_uni = biodata_all_join_therapy_uni.count()

print("Number of unique targets with GE+ & pChEMBL >= 5 & not therapeutic targets:", num_biodata_all_join_therapy_uni)

print(round(num_biodata_all_join_therapy_uni/num_biodata_all_join_targets*100), "% of targets with GE+ & pChEMBL >= 5 are not therapeutic")

Number of unique targets with GE+ & pChEMBL >= 5 & not therapeutic targets: 1090
73 % of targets with GE+ & pChEMBL >= 5 are not therapeutic


In [21]:
# Concatenate the "sources" column
biodata_all_join_uni = biodata_all_join.withColumn("sources", concat_ws(",", biodata_all_join["sources"]))
biodata_all_join_uni.repartition(1).write.csv("data/biodata_all_join_v2", header=True)